## Importing Libraries

In [ ]:
from docx2pdf import convert
from IPython.display import display, IFrame
import os
import PyPDF2 as pdf
from pyresparser import ResumeParser
import spacy
import tempfile
from docx2pdf import convert
import re
import nltk
from spacy.matcher import Matcher
import constants as cs
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from nltk.stem import WordNetLemmatizer
from pdfminer.high_level import extract_text
import streamlit as st 
import base64
import tempfile
import pythoncom

## Creating StreamLit App

In [1]:
pythoncom.CoInitialize()

def show_pdf(file_path):
    with open(file_path, "rb") as f:
        base64_pdf = base64.b64encode(f.read()).decode('utf-8')
    # pdf_display = f'<embed src="data:application/pdf;base64,{base64_pdf}" width="700" height="1000" type="application/pdf">'
    pdf_display = F'<div style="display: flex; justify-content: center;"><iframe src="data:application/pdf; base64,{base64_pdf}" width="1000" height="1000" type="application/pdf"></iframe></div>'
    st.markdown(pdf_display, unsafe_allow_html=True)

def main():
    st.set_page_config(layout="wide")
    # Set up the layout with three columns
    col1, col2, col3 = st.columns([1, 2, 1])

    # Load and display the GIF image in the left column (col1)
    gif_path = 'Graphics/Resume Parser.gif'
    with col1:
        st.image(gif_path, use_column_width=True)  # Adjust the width as needed

    # Display the text in the middle column (col2)
    with col2:
        st.markdown("<h1 style='text-align: center; font-size: 60px;'>Resume Parsing and Scoring App</h1>", unsafe_allow_html=True)

    logo_path = 'Graphics/AI Logo.png'
    # Display the logo in the right column (col3)
    with col3:
        st.image(logo_path, use_column_width=True)  # Adjust the width as needed

    pdf_file = st.file_uploader("Choose your Resume", type=["pdf", "docx"])
    text = "Hello"
    # Inputting job level
    Job_level = ["Senior management", "Middle management", "First-level management", "Experienced", "Intermediate", "Entry-level"]
    job_level_option = st.selectbox('What is your Job Level?', Job_level)
    # Inputting required job skills
    Job_skills = st.text_input("Please input a list of required skills for your job listing", "e.g Communication, MS Excel")
    # Inputting required major
    major= ["Architecture", "Biology", "Business_Administration", "Chemistry", "Criminal Justice", "Criminology", "Communications", "English", 
          "Computer_Science","Cybersecurity", "Construction_Management", "Engineering", "Military_Science", "Nursing", "Psychology", "SWAP"]
    major_option = st.selectbox("What is Major is your Job Related to?", major)

    if pdf_file is not None:
        if pdf_file.type == 'application/pdf':
            st.write("You've uploaded a PDF file.")
            # Create a temporary PDF file path
            pdf_path = tempfile.mktemp(suffix='.pdf')
            # Save the uploaded PDF file to the temporary path
            with open(pdf_path, 'wb') as f:
                f.write(pdf_file.getbuffer())
            show_pdf(pdf_path)
            text = fn.read_text_from_pdf(pdf_path)

        elif pdf_file.type == 'application/vnd.openxmlformats-officedocument.wordprocessingml.document':
            st.write("You've uploaded a DOCX file.")
            doc_path = tempfile.mktemp(suffix='.docx')
            # Save the uploaded PDF file to the temporary path
            with open(doc_path, 'wb') as f:
                f.write(pdf_file.getbuffer())
            pdf_path = tempfile.mktemp(suffix='.pdf')
            convert(doc_path, pdf_path)
            show_pdf(pdf_path)
            text = fn.read_text_from_pdf(pdf_path)
            # Uninitialize COM
            pythoncom.CoUninitialize()
    st.markdown("<h1 style='text-align: center; font-size: 60px;'>Resume Analysis</h1>", unsafe_allow_html=True)
    # Calculating Resume Score
    # Contact Info Score
    contact_info_score = 20
    if fn.extract_name(fn.nlp(text)) == None:
        contact_info_score -= 5
    if fn.extract_email(text) == None:
        contact_info_score -= 5
    if fn.extract_website(text) != None:
        contact_info_score += 5
    
    st.subheader("Contact Information\n")
    st.write(f"**First Name:** {fn.extract_name(fn.nlp(text)).split()[0]}\n")
    st.write(f"**Last Name:** {fn.extract_name(fn.nlp(text)).split()[1]}\n")
    st.write(f"**Email:** {fn.extract_email(text)}\n")
    st.write(f"**Phone No:** {fn.remove_square_brackets(fn.extract_phone_numbers(text))}\n")
    st.write(f"**Websites:** {fn.extract_website(text)}\n")
    st.subheader("Education\n")
    ed = fn.extract_education_from_resume(text)
    if ed:
        education_score = 10
        st.write(fn.extract_education_from_resume(text)[0])
    else:
        education_score = 0
        st.write("Education not detected")
    st.subheader("Objective/Summary\n")
    sections = fn.extract_entity_sections(text)
    st.subheader("Relevant Skills\n")
    relevant = fn.get_relevant_skills(major_option)
    required = fn.get_required_skills(Job_skills)
    soft = fn.get_soft_skills()
    skills = fn.extract_skills(text,required,soft,relevant)
    if skills:
        st.write(fn.extract_skills(text,required,soft,relevant))
    if "summary" in sections:
        summary_score = 10
        st.write(fn.remove_square_brackets(sections["summary"]))
    else:
        summary_score = 0
        st.write("Summary not detected")
    st.subheader("Experience\n")
    if "experience" in sections:
        experience_score = 10
        st.write(fn.remove_square_brackets(sections["experience"]))
    else:
        experience_score = 0
        st.write("Experience not detected")
    st.subheader("Projects\n")
    if "projects" in sections:
        project_score = 10
        st.write(fn.remove_square_brackets(sections["projects"]))
    else:
        project_score = 0
        st.write("Projects not detected")
    st.subheader("Honors\n")
    if "honors" in sections:
        honors_score = 10
        st.write(fn.remove_square_brackets(sections["honors"]))
    else:
        honors_score = 0
        st.write("Honors not detected")
    st.subheader("Certificates\n")
    if "certificates" in sections:
        certificates_score = 10
        st.write(fn.remove_square_brackets(sections["certificates"]))
    else:
        certificates_score = 0
        st.write("certificates not detected")
    st.subheader("Publications\n")
    if "publications" in sections:
        st.write(fn.remove_square_brackets(sections["publications"]))
    else:
        st.write("publications not detected")
    st.subheader("Resume Score\n")
    resume_score = skills_score + contact_info_score + certificates_score + honors_score + project_score + experience_score + summary_score
    st.write(resume_score)
if __name__ == "__main__":

    main()

2024-09-23 15:07:21.244 
  command:

    streamlit run C:\Users\quadr\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


## Creating List of Skills for Majors at Norwich

In [ ]:
## code modified from https://medium.com/@2020.chetaniya.bajaj/building-a-resume-parser-using-nlp-dd36c2afbce9

In [80]:
# Parsing for contact information
import re
import subprocess
# Storing a phone number
Phone_no = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
Email_address = re.compile(r'[a-zA-Z0-9\.\-+_]+@[a-zA-Z0-9\.\-+_]+\.[a-z]+')
web = re.compile(r'[a-zA-Z0-9\.\-+_]\.\+com[a-zA-Z0-9\.\-+_]')
def extract_phone_number(resume):
    phone = re.findall(Phone_no, resume)
    if resume.find(phone[0]) >= 0 and len(phone[0]) <16:
        return phone[0]
    return None
def extract_email(resume):
    email = re.findall(Email_address,resume)
    return email[0]
def extract_website(resume):
    website = re.findall(web, resume)
    return website
# Checking for email, phone_number and websites
phone_number = extract_phone_number(text)
email_address = extract_email(text)
website_link = extract_website(text)
# Calculating conctact info score
contact_info_score = 5
if len(phone_number)==0:
    contact_info_score -= 1
if len(email_address)==0:
    contact_info_score -= 1
if len(website_link)>0:
    contact_info_score+=0.5

In [87]:
# Importing NLP
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# Parsing resume for relevant skills 

def extract_skills(resume):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(resume)
 
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
 
    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
 
    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
 
    # we create a set to keep the results in.
    found_skills = set()
 
    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in skills:
            found_skills.add(token)
 
    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in skills:
            found_skills.add(ngram)

    return found_skills
skills = extract_skills(text)
# Bug: Skills are repeated
# Calculating skills score
skills_score = len(skills)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\quadr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\quadr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [114]:
# Parsing resume for educational level
EDUCATION = [
    "diploma",
    "bachelor",
    "master",
    "associate",
    "phd"]

def extract_education(resume):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(resume)
 
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
 
    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
 
    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
 
    # we create a set to keep the results in.
    found_edu = set()
 
    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in EDUCATION:
            found_edu.add(token)
    return found_edu

education_level = extract_education(text)
# Bug: Skills are repeated
# Calculating education score
education_score = 0
for edu in education_level:
    if edu.lower() == "diploma":
        education_score += 1
    elif edu.lower() == "associate":
        education_score += 2
    elif edu.lower() == "bachelor":
        education_score += 3
    elif edu.lower() == "master":
        education_score += 4
    elif edu.lower() == "ph.d":
        education_score += 5

In [188]:
# Displaying resume score
disp = f"The top ten skills for this job are: {skills}"
resume_score = contact_info_score + skills_score + education_score
display = f"Your resume score is {resume_score}:"
display_scores_1 = f"Skills: {skills_score}/10" 
display_scores_2 = f"Education: {education_score}"
display_scores_3 = f"Contact Info: {contact_info_score}"
st.write(disp)
st.write(display)
st.write(display_scores_1)
st.write(display_scores_2)
st.write(display_scores_3)